In [1]:
# configure the logger to print to console
from typing import Any
import logging
from matplotlib import pyplot as plt
import re
import random

import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV



from yeastdnnexplorer.ml_models.lasso_modeling import (
    generate_modeling_data,
    stratification_classification,
    stratified_cv_modeling,
    bootstrap_stratified_cv_modeling,
    examine_bootstrap_coefficients,
    stratified_cv_r2,
    get_interactor_importance,
    backwards_OLS_feature_selection,
    )


logging.basicConfig(level=logging.ERROR)

random.seed(42)
np.random.seed(42)

# Interactor Modeling Workflow 1

This tutorial describes a process of modeling perturbation response by binding data
with the goal of discovering a meaningful set of interactor terms. More specifically,
we start with the following model:

$$
tf_{perturbed} \sim tf_{perturbed} + tf_{perturbed}:tf_{2} + tf_{perturbed}:tf_{2} + ... + max(non\ perturbed\ binding)
$$

Where the response variable is the $tf_{perturbed}$ perturbation response, and the
predictor variables are binding data (e.g., calling card experiments). Predictor
terms such as $tf_{perturbed}:tf_{2}$ represent the interaction between the
$tf_{perturbed}$ binding and the binding of another transcription factor. The final
term, $\max(\text{non-perturbed binding})$, is defined as the maximum binding score
for each gene, excluding $tf_{perturbed}$. This term is included to mitigate the
effect of outlier genes which may have high binding scores across multiple
transcription factors, potentially distorting the model.

We assume that the actual relationship between the perturbation response and the
binding data is sparse and use the following steps to identify significant terms.
These terms represent a set of TFs which, when considered as interactors with the
perturbed TF, improve the inferred relationship between the binding and perturbation
data.


## Interactor sparse modeling

1. First, we apply bootstrapping to a 4-fold cross-validated Lasso model. The folds
are stratified based on the binding data domain of the perturbed TF, ensuring that
each fold better represents the domain structure.

    - We produce two variations of this model:
        
        1. A model trained using all available data.
        
        2. A model trained using only the top 10% of data based on the binding
        score of the perturbed TF.

1. For model `1.1`, we select coefficients whose 99.8% confidence interval does not
include zero. For model `1.2`, we select coefficients whose 90.0% confidence interval
does not include zero. We assume that, due to the non-linear relationship between
perturbation response and binding, interaction effects are more pronounced in the
top 10% of the data. By intersecting the coefficients from both models, we highlight
those that are predictive across the full dataset.

1. Now, with our reduced set of features, we perform the exact same process as Step 3 in
Workflow 1 above. That is,  With this set of predictors, next create an OLS model using the same 4-fold
stratified cross validation from which we calculated an average $R^2$. Next, for each
interactor in the model, we produce one other cross validated OLS model by
replacing the interactor with its corresponding main effect. We note if this
variant yields a better average $R^2$. We remove all features from our set in which the main effect outperforms the interactor term.

1. Finally, we report, as significant interactors, the interaction terms which have survived the steps above. We use the average R-squared achieved by this model and compare it to the average R-squared achieved by the univariate counterpart (the response TF predicted solely by its main effect). We would like to create a boxplot of this comparison across all TFs to see how this pipeline affects model performance in explaining variance in contrast to the simple univariate model.


***NOTE***: To generate the `response_df` and `predictors_df` below, see the first six 
cells in the LassoCV tutorial.

In [2]:
response_df = pd.read_csv("~/htcf_local/lasso_bootstrap/erics_tfs/response_dataframe_20241105.csv", index_col=0)
predictors_df = pd.read_csv("~/htcf_local/lasso_bootstrap/erics_tfs/predictors_dataframe_20241105.csv", index_col=0)

### Step 1: Find significant predictors using all of the data

The function `get_significant_predictors()` is a wrapper of the lassoCV bootstrap 
protocol described in the LassoCV notebook. It allows using the same code to produce
both the 'all data' (step 1.1) and 'top 10%' models (step 1.2), and returns the 
significant coefficients as described in the protocol above.

In [3]:

# TODO: the top10% models likely should not have teh same number of classes as the
# all data, possibly not stratified at all

def get_significant_predictors(
    perturbed_tf: str,
    response_df: pd.DataFrame,
    predictors_df: pd.DataFrame,
    add_max_lrb: bool,
    **kwargs: Any,
) -> tuple[dict[str, tuple[float, float]], pd.DataFrame, pd.Series]:
    """
    This function is used to get the significant predictors for a given TF. It is
    capable of conducting steps 1.1 and 1.2 described above.

    :param perturbed_tf: the TF for which the significant predictors are to be
        identified
    :param response_df: The DataFrame containing the response values
    :param predictors_df: The DataFrame containing the predictor values
    :param add_max_lrb: A boolean to add/not add in the max_LRB term for a response TF
        into the formula that we perform bootstrapping on
    :param kwargs: Additional arguments to be passed to the function. Expected arguments
        are 'quantile_threshold' from generate_modeling_data() and 'ci_percentile' from
        examine_bootstrap_coefficients()
    :return sig_coef_dict: A dictionary containing the significant predictors and their
        corresponding coefficients

    """

    y, X = generate_modeling_data(
        perturbed_tf,
        response_df,
        predictors_df,
        quantile_threshold=kwargs.get("quantile_threshold", None),
        drop_intercept=True,
    )

    # NOTE: fit_intercept is set to `true`
    lassoCV_estimator = LassoCV(
        fit_intercept=True,
        max_iter=10000,
        selection="random",
        random_state=42,
        n_jobs=4,
    )

    predictor_variable = re.sub(r"_rep\d+", "", perturbed_tf)

    stratification_classes = stratification_classification(
        X[predictor_variable].squeeze(), y.squeeze()
    )

    # Fit the model to the data in order to extract the alphas_ which are generated
    # during the fitting process
    lasso_model = stratified_cv_modeling(
        y, X, stratification_classes, lassoCV_estimator
    )

    if add_max_lrb:
        # add a column to X which is the rowMax excluding column `predictor_variable`
        # called max_lrb
        X["max_lrb"] = X.drop(columns=predictor_variable).max(axis=1)

    # set the alphas_ attribute of the lassoCV_estimator to the alphas_ attribute of the
    # lasso_model fit on the whole data. This will allow the
    # bootstrap_stratified_cv_modeling function to use the same set of lambdas
    lassoCV_estimator.alphas_ = lasso_model.alphas_

    # for test purposes, set n_bootstraps to 10
    # NOTE: fit_intercept=True is passed to the internal Lasso model for bootstrap
    # iterations, along with some other settings

    logging.info("running bootstraps")
    bootstrap_lasso_output = bootstrap_stratified_cv_modeling(
        y=y,
        X=X,
        estimator=lassoCV_estimator,
        ci_percentile=kwargs.get("ci_percentile", 95.0),
        n_bootstraps=kwargs.get("n_bootstraps", 10),
        max_iter=10000,
        fit_intercept=True,
        selection="random",
        random_state=42,
    )

    sig_coef_plt, sig_coef_dict = examine_bootstrap_coefficients(
        bootstrap_lasso_output, ci_level=kwargs.get("ci_percentile", 95.0)
    )

    plt.close(sig_coef_plt)

    return sig_coef_dict, y, stratification_classes



In [4]:
all_data_sig_coef, all_y, all_stratification_classes = get_significant_predictors(
    "CBF1",
    response_df,
    predictors_df,
    ci_percentile=99.8,
    n_bootstraps=100,
    add_max_lrb=True)

top10_data_sig_coef, top10_y, top10_stratification_classes = get_significant_predictors(
    "CBF1",
    response_df,
    predictors_df,
    quantile_threshold=0.1,
    ci_percentile=90.0,
    n_bootstraps=100,
    add_max_lrb=True)



Significant coefficients for 99.8, where intervals are entirely above or below ±0.0:
CBF1:SWI6: (-0.14416455697760805, -0.009884405236493745)
CBF1:RGM1: (0.014576695345595829, 0.1606224517888866)
CBF1:ARG81: (-0.2148742948328769, -0.03345111234665489)
CBF1:MET28: (0.07821562304993965, 0.20784574601671207)
CBF1:AZF1: (-0.1541657715595909, -0.026421884027885357)
CBF1:GAL4: (0.09086284981352442, 0.31851294627923615)
CBF1:MSN2: (0.10065808506838021, 0.27766726330427544)
max_lrb: (0.0018092836372706894, 0.0983814591861912)
Significant coefficients for 90.0, where intervals are entirely above or below ±0.0:
CBF1:MET28: (0.06998122210410228, 0.2088974100580522)


## Step 2

We next need to intersect the significant coefficients (see definitions above) in both
models. In this case, one interactors survives (note that there are only 100
bootstraps in this example in the interest of speed for the tutorial. We recommend no 
less than 1000 in practice).

In [5]:
intersect_coefficients = set(all_data_sig_coef.keys()).intersection(set(top10_data_sig_coef.keys()))
print(f"The surviving coefficients are: {intersect_coefficients}")

The surviving coefficients are: {'CBF1:MET28'}


## Step 3

We next implement the method which searches alternative models, which include the
surviving interactor terms, with variations on substituing in the main effect. In this case, 
we have a single term. However, if we had more than one term, we would do the following for each surviving interactor term. The goal of this process, remember, is to generate a set of
high confidence interactor terms for this TF. If the predictive power of the main effect
is equivalent or better than a model with the interactor, we consider that a low
confidence interactor effect. 

After identifying all interactor terms for which substituting in the main effect improves
the average R-squared from CV, we drop these terms from our set of features. We then log the final average R-squared achieved by this model. In this case, no terms are dropped from testing the substitution of main effects.

In [6]:

# get the additional main effects which will be tested from the intersect_coefficients
main_effects = []
for term in intersect_coefficients:
    if ":" in term:
        main_effects.append(term.split(":")[1])
    else:
        main_effects.append(term)

# combine these main effects with the intersect_coefficients
interactor_terms_and_main_effects = list(intersect_coefficients) + main_effects

# generate a model matrix with the intersect terms and the main effects. This full
# model will not be used for modeling -- subsets of the columns will be, however.
_, full_X = generate_modeling_data(
    'CBF1',
    response_df,
    predictors_df,
    formula = f"~ {' + '.join(interactor_terms_and_main_effects)}",
    drop_intercept=False,
)

full_X["max_lrb"] = predictors_df.drop(columns="CBF1").max(axis=1)

# Currently, this function tests each interactor term in the intersect_coefficients
# with two variants by replacing the interaction term with the main effect only, and
# with the main effect + interactor. If either of the variants has a higher avg
# r-squared than the intersect_model, then that variant is returned. In this case,
# the original intersect_coefficients are the best model.
full_avg_rsquared, x = get_interactor_importance(
    all_y,
    full_X,
    all_stratification_classes,
    intersect_coefficients
)

print(f"The full model avg r-squared is {full_avg_rsquared}")
print(f"The interactor results are: {x}")

# Now that we have identifed the interactors whose main effects improve average R-squared, we drop them from our model

if x:
    interactors_to_remove = set()
    for dictionary in x:
        interactor = dictionary.get("interactor")
        interactors_to_remove.add(interactor)  
        print("Removing term: "+str(interactor))

    final_feature_set = intersect_coefficients.difference(interactors_to_remove)
    # get the final avg r-squared for this set
    final_model_avg_r_squared, _ = get_interactor_importance(
        all_y,
        full_X,
        all_stratification_classes,
        final_feature_set
    )

else:
    final_feature_set = intersect_coefficients
    final_model_avg_r_squared = full_avg_rsquared

print(f"The final model avg r-squared is {final_model_avg_r_squared}")
print(f"Final set of terms: {final_feature_set}")

The full model avg r-squared is 0.01051720848723997
The interactor results are: []
The final model avg r-squared is 0.01051720848723997
Final set of terms: {'CBF1:MET28'}


/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


## Step 4: Comparing our final model to a univariate model

In our last step, we take our reamining set of features from the end of Step 3, and now compare its performance to that of a univariate model where the response TF is predicted solely by its main effect. We will use the average R-squared achieved by 4-Fold CV on both models.

In [7]:
y, X = generate_modeling_data("CBF1", response_df, predictors_df, drop_intercept=True, formula="CBF1_LRR ~ CBF1")
classes = stratification_classification(X["CBF1"].squeeze(), y.squeeze())
avg_r2_univariate = stratified_cv_r2(
        y,
        X,  
        classes,
    )

print(f"The univariate average R-squared is: {avg_r2_univariate}")
print(f"The final model average R-squared is {final_model_avg_r_squared}")

The univariate average R-squared is: 0.004970412632932103
The final model average R-squared is 0.01051720848723997


/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


As we can see, the final model we achieved through Workflow 1 demonstrates a higher average R-squared achieved by 4-fold CV. 

# Interactor Modeling Workflow 2

An alternative workflow to identifying a meaningful set of transcription factors takes a slightly different approach. There are some commonalities between this workflow and Workflow 1, and we will point them out in the steps below which outline how this alternative workflow operates.


1. First, we apply a 4-fold cross-validated Lasso model (without bootstrapping - this is the 
difference between this step and Step 1 from Workflow 1). The folds
are stratified based on the binding data domain of the perturbed TF, ensuring that
each fold better represents the domain structure.

    - We produce two variations of this model:
        
        1. A model trained using all available data.
        
        2. A model trained using only the top 10% of data based on the binding
        score of the perturbed TF.

2. Each Lasso model from Step 1 will return a set of non-zero coefficients. We then intersect the coefficients from both models, and retain this list of features
in the exact same fashion as Step 2 of Workflow 1.  

3. With this set of predictors, we then perform a "backwards OLS feature selection," in 
which we create OLS models using this set of predictors on the entire dataset, and remove
features which have a p-value above a threshold for significance (0.001). We continously 
remove features and re-create models on the reduced set of features until all of the 
features in a model are significant. Then, taking this filtered set of predictors, we 
perform the same process, but this time on the top 10% of the data based on the binding
score of the perturbed TF. Since we are now using a smaller dataset, our threshold for 
significance is increased (0.01), and we perform the same process until we arrive at a 
final model in which all of the terms are significant. 

From here onwards, we follow the same steps as Step 3 and Step 4 in Workflow 1 above. I have copied their descriptions from above, and have renamed them Steps 4 and 5 to match the numbering for this workflow.

4. Now, with our reduced set of features, we perform the exact same process as Step 3 in
Workflow 1 above. That is,  With this set of predictors, next create an OLS model using the same 4-fold
stratified cross validation from which we calculated an average $R^2$. Next, for each
interactor in the model, we produce one other cross validated OLS model by
replacing the interactor with its corresponding main effect. We note if this
variant yields a better average $R^2$. We remove all features from our set in which the main effect outperforms the interactor term.

5. Finally, we report, as significant interactors, the interaction terms which have survived the steps above. We use the average R-squared achieved by this model and compare it to the average R-squared achieved by the univariate counterpart (the response TF predicted solely by its main effect). We would like to create a boxplot of this comparison across all TFs to see how this pipeline affects model performance in explaining variance in contrast to the simple univariate model.

Let's choose a particular TF to run though this workflow with.

In [3]:
tf_of_interest = "CBF1"

## Step 1: get the non-zero coefficients from the Lasso models

The function `get_non_zero_predictors()` is a wrapper of the stratified CV modeling 
protocol described in the LassoCV notebook. It allows using the same code to produce
the Lasso models trained on the 'all data' (step 1.1) and 'top 10%' models (step 1.2), and returns the 
features with non-zero coefficients as described in the protocol above.

In [4]:

def get_non_zero_predictors(
    perturbed_tf: str,
    response_df: pd.DataFrame,
    predictors_df: pd.DataFrame,
    **kwargs: Any,
) -> list[str]:
    """
    This function is used to get the features with non-zero coefficients from LassoCV
    for a given TF. It is capable of conducting steps 1a and 1b above.

    :param perturbed_tf: str, the TF for which the significant predictors are to be
        identified
    :param response_df: pd.DataFrame, the response dataframe containing the response
        values
    :param predictors_df: pd.DataFrame, the predictors dataframe containing the
        predictor values
    :param kwargs: dict, additional arguments to be passed to the function. Expected
        arguments is 'quantile_threshold' fom generate_modeling_data() to signify the
        top 10%
    :return non_zero_features: List, a list containing the features with non-zero coefs

    """
    # Validate input
    if perturbed_tf not in response_df.columns:
        raise ValueError(
            f"The response TF {perturbed_tf} does not exist in the response DataFrame."
        )
    if perturbed_tf not in predictors_df.columns:
        raise ValueError(
            f"The response TF {perturbed_tf} does not exist in the binding DataFrame."
        )
    if response_df.shape[0] != predictors_df.shape[0]:
        raise ValueError(
            "The binding and response DataFrames contain different counts of genes"
        )

    tf_y, tf_X = generate_modeling_data(
        perturbed_tf,
        response_df,
        predictors_df,
        quantile_threshold=kwargs.get("quantile_threshold", None),
        drop_intercept=True,
    )

    # add the max_lrb term to the formula
    tf_X["max_lrb"] = predictors_df.drop(columns=perturbed_tf).max(axis=1)

    # NOTE: fit_intercept is set to `true`
    lassoCV_estimator = LassoCV(
        fit_intercept=True,
        max_iter=10000,
        selection="random",
        random_state=42,
        n_jobs=4,
    )

    # create the classifications
    classes = stratification_classification(tf_X[perturbed_tf], tf_y[perturbed_tf])

    # Fit the model to the data in order to extract the non-zero coefficients
    # surviving after the fitting process
    lasso_model = stratified_cv_modeling(tf_y, tf_X, classes, lassoCV_estimator)

    # return a list of the non-zero features that survived the fitting
    non_zero_indices = lasso_model.coef_ != 0
    non_zero_features = tf_X.columns[non_zero_indices]

    return non_zero_features

In [5]:
# Step 1.1: get the non-zero features on all data
tf_surviving_terms = get_non_zero_predictors(tf_of_interest, response_df, predictors_df)
# Step 1.2: get the non-zero features on only the top10% of genes
tf_top10_surviving_terms = get_non_zero_predictors(tf_of_interest, response_df, predictors_df, quantile_threshold=0.1)

## Step 2: Intersect the features from both Lasso models

Here, we simply intersect the sets of non-zero features found by both of the Lasso models in Step 1.

In [6]:
intersect_coefficients = set(tf_surviving_terms).intersection(set(tf_top10_surviving_terms))
print(f"The surviving coefficients are: {intersect_coefficients}")

The surviving coefficients are: {'CBF1:SWI6', 'CBF1:SKN7', 'CBF1:AZF1', 'CBF1:MET28'}


## Step 3: Perform backwards OLS feature selection on the intersected features

Now, taking our set of intersecting features from the step above, we perform the process of backwards OLS feature selection as described above. The function `backwards_OLS_feature_selection()` is a wrapper function that repeatedly calls `select_significant_features()` to perform the iterative process of removing insignificant features based on their p-value with respect to the given quantile threshold. Currently, we are performing this both on the entire dataset with a p-value threshold of 0.001, then subsequently on the top 10% by perturbed TF binding data with a p-value threshold of 0.01.

In [7]:
quantile_thresholds = [None, 0.1]  # None for full dataset, 0.1 for top 10%
p_value_thresholds = [0.001, 0.01]  # Corresponding p-value thresholds for both datasets

backward_OLS_feature_result = backwards_OLS_feature_selection(
    tf_of_interest,
    intersect_coefficients,
    response_df,
    predictors_df,
    quantile_thresholds,
    p_value_thresholds,
)

print(f"The surviving features from backwards OLS feature selection is {backward_OLS_feature_result}")

The surviving features from backwards OLS feature selection is {'CBF1:MET28', 'CBF1:SWI6'}


## Step 4

This is now the exact same workflow as Step 3 from Workflow 1. To recap, we now implement the method which searches alternative models, which include the surviving interactor terms, with variations on including the main effect. The goal of this process is to generate a set of high confidence interactor terms for this TF. If the predictive power of the main effect is equivalent or better than a model with the interactor, we consider that a low confidence interactor effect.

In [8]:

# get the additional main effects which will be tested from the backward_OLS_feature_result
main_effects = []
for term in backward_OLS_feature_result:
    if ":" in term:
        main_effects.append(term.split(":")[1])
    else:
        main_effects.append(term)

# combine these main effects with the backward_OLS_feature_result
interactor_terms_and_main_effects = list(backward_OLS_feature_result) + main_effects

# generate a model matrix with the intersect terms and the main effects. This full
# model will not be used for modeling -- subsets of the columns will be, however.
_, full_X = generate_modeling_data(
    tf_of_interest,
    response_df,
    predictors_df,
    formula = f"~ {' + '.join(interactor_terms_and_main_effects)}",
    drop_intercept=False
)

# have to generate the stratification classes and the "y" column for input into 
# get_interactor_importance below
y, X = generate_modeling_data(tf_of_interest, response_df, predictors_df)
all_stratification_classes = stratification_classification(X[tf_of_interest].squeeze(), y.squeeze())

# Currently, this function tests each interactor term in the backward_OLS_feature_result
# with two variants by replacing the interaction term with the main effect only, and
# with the main effect + interactor. If either of the variants has a higher avg
# r-squared than the intersect_model, then that variant is returned. 
full_avg_rsquared, x = get_interactor_importance(
    y,
    full_X,
    all_stratification_classes,
    backward_OLS_feature_result
)

print(f"The full model avg r-squared is {full_avg_rsquared}")
print(f"The interactor results are: {x}")

if x:
    interactors_to_remove = set()
    for dictionary in x:
        interactor = dictionary.get("interactor")
        interactors_to_remove.add(interactor)  
        print(f"Removing term: {interactor}")

    final_feature_set = backward_OLS_feature_result.difference(interactors_to_remove)
    # get the final avg r-squared for this set
    final_model_avg_r_squared, _ = get_interactor_importance(
        y,
        full_X,
        all_stratification_classes,
        final_feature_set
    )

else:
    final_feature_set = backward_OLS_feature_result
    final_model_avg_r_squared = full_avg_rsquared

print(f"The final model avg r-squared is {final_model_avg_r_squared}")
print(f"Final set of terms: {final_feature_set}")

/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


The full model avg r-squared is 0.016951766474153418
The interactor results are: []
The final model avg r-squared is 0.016951766474153418
Final set of terms: {'CBF1:MET28', 'CBF1:SWI6'}


## Step 5

This is the same as Step 4 from Workflow 1 above. 

In [9]:
y, X = generate_modeling_data(tf_of_interest, response_df, predictors_df, drop_intercept=True, formula=f"{tf_of_interest}_LRR ~ {tf_of_interest}")
avg_r2_univariate = stratified_cv_r2(
        y,
        X,  
        all_stratification_classes,
    )

print(f"The univariate average R-squared is: {avg_r2_univariate}")
print(f"The final model average R-squared is {final_model_avg_r_squared}")

The univariate average R-squared is: 0.004970412632932103
The final model average R-squared is 0.016951766474153418


/Users/ericjia/yeastdnnexplorer/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


As we can see, the final mdel we achieved through Workflow 2 demonstrates a higher average R-squared achieved by 4-fold CV. 